<a href="https://colab.research.google.com/github/cccadet/dermatologist-ai/blob/master/01_Tflite_Ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Pré: Conexão com Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Etapa 1: Instalação das dependências

In [2]:
!pip install tensorflow-gpu

### Importação das bibliotecas

In [3]:
import zipfile
import os
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa

import matplotlib.pyplot as plt
import pandas as pd
import sys
import itertools
from keras_radam import RAdam
import shutil

from tqdm import tqdm_notebook
from tensorflow.keras.preprocessing.image import ImageDataGenerator

%matplotlib inline
tf.__version__

'2.3.0'

### Otimizações tensorflow

In [4]:
#from tensorflow.keras.mixed_precision import experimental as mixed_precision
#policy = mixed_precision.Policy('mixed_float16')
#mixed_precision.set_policy(policy)
tf.config.optimizer.set_jit(True)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

### Configurando os caminhos (paths)

In [5]:
download_path = './'
derm_ai_path = '/content/drive/My Drive/Derm AI/ISIC_2019'

### Salvar modelo após Fine Tuning

In [6]:
%%time
print("Model1")
model1 = tf.keras.models.load_model(os.path.join(derm_ai_path, 'model1_B7'))
print("Model2")
model2 = tf.keras.models.load_model(os.path.join(derm_ai_path, 'model2_InceptionResnetV2'))
print("Model3")
model3 = tf.keras.models.load_model(os.path.join(derm_ai_path, 'model3_NasNet_Large'))
print("Model4")
model4 = tf.keras.models.load_model(os.path.join(derm_ai_path, 'model4_InceptionV3'))
print("Model5")
model5 = tf.keras.models.load_model(os.path.join(derm_ai_path, 'model5_Xception'))
print("Model6")
model6 = tf.keras.models.load_model(os.path.join(derm_ai_path, 'B1-IRN50V2_3'))

Model1
Model2
Model3
Model4
Model5
Model6
CPU times: user 8min 56s, sys: 22.4 s, total: 9min 18s
Wall time: 10min 42s


In [7]:
model1._name="model1"
model2._name="model2"
model3._name="model3"
model4._name="model4"
model5._name="model5"
model6._name="model6"

In [8]:
#personalização e união
x = tf.keras.layers.Input(shape=[299, 299, 3])

model_1 = model1(x)
model_1 = tf.math.multiply(model_1, [0.0067])

model_2 = model2(x)
model_2 = tf.math.multiply(model_2, [0.2621])

model_3 = model3(x)
model_3 = tf.math.multiply(model_3, [0.0019])

model_4 = model4(x)
model_4 = tf.math.multiply(model_4, [0.0205])

model_5 = model5(x)
model_5 = tf.math.multiply(model_5, [0.0510])

model_6 = model6(x)
model_6 = tf.math.multiply(model_6, [0.6577])

base_model1 = tf.math.add(model_1, model_2)
base_model2 = tf.math.add(base_model1, model_3)
base_model3 = tf.math.add(base_model2, model_4)
base_model4 = tf.math.add(base_model3, model_5)
prediction_layer =  tf.math.add(base_model4, model_6)

#prediction_layer = tf.keras.layers.Average()([model_1, model_2, model_4, model_5])

#prediction_layer = tf.keras.layers.Dense(units = 8, activation = "softmax")(base_model)

#final_model
model = tf.keras.models.Model(inputs = x, outputs = prediction_layer)

#optimizer
radam = tfa.optimizers.RectifiedAdam(
    lr=1e-2,
    total_steps=251,
    warmup_proportion=0.1,
    min_lr=1e-5)
ranger = tfa.optimizers.Lookahead(radam, sync_period=6, slow_step_size=0.5)
optm = ranger

#compilador
model.compile(optimizer=optm, loss=tf.keras.losses.CategoricalCrossentropy(), metrics=[tf.keras.metrics.CategoricalAccuracy()])

#model summary
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
model1 (Functional)             (None, 8)            64118175    input_1[0][0]                    
__________________________________________________________________________________________________
model2 (Functional)             (None, 8)            54349032    input_1[0][0]                    
__________________________________________________________________________________________________
tf_op_layer_Mul (TensorFlowOpLa [(None, 8)]          0           model1[0][0]                     
_______________________________________________________________________________________

In [ ]:
model.save(os.path.join(derm_ai_path, 'ensemble'))


```
"AK":pred[:,0],
"BCC":pred[:,1],
"BKL":pred[:,2],
"DF":pred[:,3],
"MEL":pred[:,4],
"NV":pred[:,5],
"SCC":pred[:,6],
"VASC":pred[:,7],
```



### Salvar modelo com score obtido

### Carregar melhor modelo 

## Etapa 5: Tensorflow Lite

### Converter modelo para formato tflite

In [9]:
%%time
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)

CPU times: user 36 µs, sys: 2 µs, total: 38 µs
Wall time: 40.8 µs


In [ ]:
%%time
#converter.post_training_quantize=True
tflite_model = converter.convert()

### Salvar arquivo convertido

In [ ]:
%%time
model_tflite = os.path.join(derm_ai_path, 'Ensemble.tflite')
open(model_tflite, "wb").write(tflite_model)

In [ ]:
%%time
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
tflite_model_quantization = converter.convert()

In [ ]:
%%time
model_tflite_quantization = os.path.join(derm_ai_path, 'Ensemble_quantization.tflite')
open(model_tflite_quantization, "wb").write(tflite_model_quantization)

### Testar modelo convertido (TFLite)
https://medium.com/towards-artificial-intelligence/testing-tensorflow-lite-image-classification-model-e9c0100d8de3

https://colab.research.google.com/github/frogermcs/TFLite-Tester/blob/master/notebooks/Testing_TFLite_model.ipynb#scrollTo=6WEyRJNsR5uL

https://aihub.cloud.google.com/p/products%2F6ca92c9c-6932-45f8-8f08-15a1a6dfafc7

In [ ]:
tflite_interpreter = tf.lite.Interpreter(model_path=InceptionResNetV2_tflite)

input_details = tflite_interpreter.get_input_details()
output_details = tflite_interpreter.get_output_details()

print("== Input details ==")
print("shape:", input_details[0]['shape'])
print("type:", input_details[0]['dtype'])
print("\n== Output details ==")
print("shape:", output_details[0]['shape'])
print("type:", output_details[0]['dtype'])

== Input details ==
shape: [  1 299 299   3]
type: <class 'numpy.float32'>

== Output details ==
shape: [1 3]
type: <class 'numpy.float32'>


In [ ]:
tflite_interpreter.resize_tensor_input(input_details[0]['index'], (32, 299, 299, 3))
tflite_interpreter.resize_tensor_input(output_details[0]['index'], (32, 3))
tflite_interpreter.allocate_tensors()

input_details = tflite_interpreter.get_input_details()
output_details = tflite_interpreter.get_output_details()

print("== Input details ==")
print("shape:", input_details[0]['shape'])
print("\n== Output details ==")
print("shape:", output_details[0]['shape'])

== Input details ==
shape: [ 32 299 299   3]

== Output details ==
shape: [32  3]


In [ ]:
test_generator.reset()
#pred=model.predict_generator(test_generator,verbose=1)

In [ ]:
test_image_batch, test_label_batch = next(iter(test_generator))
true_label_ids = np.argmax(test_label_batch, axis=-1)

print("Validation batch shape:", test_image_batch.shape)

Validation batch shape: (32, 299, 299, 3)


In [ ]:
tflite_interpreter.set_tensor(input_details[0]['index'], test_image_batch)

tflite_interpreter.invoke()

tflite_model_predictions = tflite_interpreter.get_tensor(output_details[0]['index'])
print("Prediction results shape:", tflite_model_predictions.shape)

Prediction results shape: (32, 3)


In [ ]:
tflite_model_predictions

In [ ]:
results_file_tflite = os.path.join(derm_ai_path, 'results_tflite_InceptionResNetV2_full_sp.csv')

saveResultsFile(results_file_tflite, tflite_model_predictions, test_generator)

ValueError: ignored